### Libraries

In [ ]:
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import os
import statsmodels.api as sm
from pandas.tseries.offsets import DateOffset

# Changing Current Directory
print('Current Working Directory : ',os.getcwd())
if str(os.getcwd()).split("/")[-1] != "DrOetkar - SARIMAX":
    os.chdir("../Projects/DrOetkar - SARIMAX/")
    print('Changed to : ',os.getcwd())

### Prepare Dataset

In [ ]:
df=pd.read_excel('sales2.xlsx')

# Create week var
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["week_nu"] = df["Order Date"].dt.year.astype(str) + 'W' + df['Week'].astype(str)

sumdf=pd.DataFrame(df.groupby(["Description", "week_nu"]).agg({'Order Date':'last', 'QTY':'sum'}).reset_index())
sumdf = sumdf.sort_values(by=['Description','Order Date'], ascending=True)
df_head = sumdf.set_index('Order Date').copy()

# Transposed DS for later use
df_head_trans = pd.pivot_table(df_head, values ='QTY', index =['Description'],
                         columns =['week_nu'], aggfunc = np.sum)


#passed other tests and defined - ADF, PACF, ACF ; p,d,q
# Filter for SKU -  Pasta Pizza Sauce 325g
df_head=df_head.loc[df_head['Description']=='Pasta Pizza Sauce 325g']
print(df_head.Description.value_counts())

### Prepare Model

#### Testing on train

In [ ]:
# List of items
sku_list = df_head.Description.unique()
df_forecast = pd.DataFrame([])

for i in sku_list:
    # Create a folder for graphs
    if not os.path.exists('Graphs'):
        os.makedirs('Graphs')

    ### Common Part
    df_sku = df_head.loc[df_head.Description==i, :]
    model=sm.tsa.statespace.SARIMAX(df_sku['QTY'], order=(1, 1, 1),seasonal_order=(1,1,1,12))
    results=model.fit()
    
    ### Testing on train
    df_sku['forecast']=results.predict(start='2022-03-05',end ='2022-03-31',dynamic=True)
    df_sku[['QTY','forecast']].plot(figsize=(25,5))
    plt.title(i)
    plt.savefig('./Graphs/' + i+'.png')

    ### Forecast and Display
    # Add future dates
#     future_dates=[df_sku.index[-1]+ DateOffset(weeks=x)for x in range(0,10)]
#     future_datest_df=pd.DataFrame(index=future_dates[1:],columns=df_sku.columns)
#     future_df=pd.concat([df_sku,future_datest_df])
       
#     # Forecasts
#     future_df = future_df.reset_index()
#     future_df["forecast"] = results.forecast(steps=100)

#     # final df appended
#     _tmp = future_df.copy()
#     _tmp['Description'] = i
#     _tmp.loc[_tmp.forecast.isna(), 'forecast'] = _tmp['QTY']
#     df_forecast = df_forecast.append(_tmp)

#     # Plot
#     future_df = future_df.set_index("index")
#     future_df[['QTY', 'forecast']].plot(figsize=(25, 5))
#     plt.savefig('./Graphs/' + i+'.jpeg', bbox_inches='tight', transparent=False, dpi=300)
#     plt.title(i)
#     plt.show()

# df_forecast.to_csv("./Graphs/forecasted_data.csv", index=False)